<a href="https://colab.research.google.com/github/rafaelhamer/DSWP/blob/master/Kaggle_Labdata_Churn_Z1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost

     |████████████████████████████████| 66.2MB 55kB/s 


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#from sklearn.metrics import confusion_matrix # para plotar a confusion matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

def mostra_missing_value(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = 100*round((df.isnull().sum()/df.isnull().count()).sort_values(ascending = False), 2)
    missing_data = pd.concat([total, percent], axis = 1, keys=['Total', 'Percentual'])
    print(missing_data.head(10))

from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import os
for dirname, _, filenames in os.walk('/gdrive/My Drive/Python'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Lendo os dados de Treino e Teste

In [77]:
df_train = pd.read_csv('/gdrive/My Drive/Python/train.csv')#, index_col='id')
df_test  = pd.read_csv('/gdrive/My Drive/Python/test.csv')#, index_col='id')

In [ ]:
df_train.head()

,id,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,NaN,45.05,2560.1,0
1,6731,Male,0,Yes,Yes,NaN,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0
2,6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.7,0
3,6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1
4,3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.8,1


In [ ]:
df_train.isna().sum()

In [ ]:
df_train.dtypes

In [75]:
class TrataDataframe():
  def __init__(self,df):
      self.df = df
      self.trata_Pagamentos()
      self.trata_Numeros()
      self.trata_Servicos()
      self.trata_Dependentes()  
      self.trata_MetodoPagamento()

  #df_train.drop('gender', axis=1, inplace=True)

  def trata_Pagamentos(self):
    self.df['TotalCharges'] = self.df['TotalCharges'].str.replace(' ','0.0').astype(np.float64)
    self.df['tenure'] = self.df['tenure'].fillna(self.df['TotalCharges'] / self.df['MonthlyCharges'])
    self.df.drop('TotalCharges', axis=1, inplace=True)
    #self.df['CobrancaTotal'] = pd.to_numeric(self.df['CobrancaTotal'], errors='coerce')

  def trata_Numeros(self):
    self.df['MonthlyCharges'] = self.df['MonthlyCharges'].astype(np.int16)
    self.df['tenure'] = self.df['tenure'].astype(np.int16)

  def trata_Servicos(self):
    #self.df['MultipleLines'] = self.df['MultipleLines'].replace({'No phone service':'No'})
    self.df.drop('PhoneService', axis=1, inplace=True)
    self.df['InternetService'] = self.df['InternetService'].replace({'No internet service':'No'})
    self.df['OnlineSecurity'] = self.df['OnlineSecurity'].replace({'No internet service':'No'})
    self.df['OnlineBackup'] = self.df['OnlineBackup'].replace({'No internet service':'No'})
    self.df['DeviceProtection'] = self.df['DeviceProtection'].replace({'No internet service':'No'})
    self.df['TechSupport'] = self.df['TechSupport'].replace({'No internet service':'No'})
    self.df['StreamingTV'] = self.df['StreamingTV'].replace({'No internet service':'No'})
    self.df['StreamingMovies'] = self.df['StreamingMovies'].replace({'No internet service':'No'})

  def trata_Dependentes(self):
    self.df['Dependents'].fillna('MISS', inplace=True)
    self.df.loc[(self.df['Partner'] == 'No') & (self.df['Dependents'] == 'MISS'), 'Dependents'] = "No"

  def trata_MetodoPagamento(self):
    self.df['PaymentMethod'].fillna('MISS', inplace=True)
    #self.df['PaymentMethod'].fillna('Electronic check', inplace=True)
    #self.df = self.df.dropna(axis = 0, subset = ['MetodoPagamento'])

  def getDataFrameDefault(self):
        return self.df  

  def getDataFrameMelhoresFeatures(self):
        ''' retorna o dataframe default, adicionando duas novas colunas (qtdeServicos e vlrMedioPorServico), e igualando "No" e "No internet service" das colunas relativas a serviços online '''
        df2 = self.df.copy()
        df2["QtdeServicosAdicionaisInternet"] = 0
        df2.loc[df2["ServicoSegurancaOnline"] == "Yes","QtdeServicosAdicionaisInternet"] = df2["QtdeServicosAdicionaisInternet"] + 1
        df2.loc[df2["ServicoBackupOnline"] == "Yes","QtdeServicosAdicionaisInternet"] =  df2["QtdeServicosAdicionaisInternet"] + 1
        df2.loc[df2["ServicoProtecaoOnline"] == "Yes","QtdeServicosAdicionaisInternet"] =  df2["QtdeServicosAdicionaisInternet"] + 1
        df2.loc[df2["ServicoSuporteOnline"] == "Yes","QtdeServicosAdicionaisInternet"] = df2["QtdeServicosAdicionaisInternet"] + 1

        df2.loc[df2["ServicoTV"] != "Yes","ServicoTV"] =  "No"
        df2.loc[df2["ServicoPctFilmes"] != "Yes","ServicoPctFilmes"] = "No"
        df2["QtdeServicosAdicionaisInternet"] = df2["QtdeServicosAdicionaisInternet"].astype('float')
        df2.drop(columns=["genero","TemParceiro","TemTelefone","ServicoSegurancaOnline","ServicoBackupOnline","ServicoProtecaoOnline","ServicoSuporteOnline" ],inplace=True)
        return df2         

  def getDataFramePrecoMedio(self):
        ''' retorna o dataframe default, adicionando duas novas colunas (qtdeServicos e vlrMedioPorServico), e igualando "No" e "No internet service" das colunas relativas a serviços online '''
        df2 = self.df.copy()
        df2["QtdeServicos"] = 0
        df2.loc[df2["TemTelefone"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["TemInternet"] != "No","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoTV"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoSegurancaOnline"] == "Yes","QtdeServicos"] = df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoBackupOnline"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoProtecaoOnline"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoPctFilmes"] == "Yes","QtdeServicos"] = df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoSuporteOnline"] == "Yes","QtdeServicos"] = df2["QtdeServicos"] + 1

        df2.loc[df2["ServicoTV"] != "Yes","ServicoTV"] =  "No"
        df2.loc[df2["ServicoSegurancaOnline"] != "Yes","ServicoSegurancaOnline"] = "No"
        df2.loc[df2["ServicoBackupOnline"] != "Yes","ServicoBackupOnline"] =  "No"
        df2.loc[df2["ServicoProtecaoOnline"] != "Yes","ServicoProtecaoOnline"] = "No"
        df2.loc[df2["ServicoPctFilmes"] != "Yes","ServicoPctFilmes"] = "No"
        df2.loc[df2["ServicoSuporteOnline"] != "Yes","ServicoSuporteOnline"] = "No"
        df2["QtdeServicos"] = df2["QtdeServicos"].astype('float')
        df2.drop(columns=["TemParceiro"],inplace=True)
        #df2["vlrMedioPorServico"] = df2["CobrancaMensal"] / df2["QtdeServicos"]

        return df2  

  def getDataFrameServAdicionais(self):
        ''' gera um dataframe com as transformações "padrão"...e cria uma nova coluna apenas com o indicativo pra saber se há serviços adicionais de internet '''
        df2 = self.df.copy()
        df2["ServicosAdicionaisInternet"] = "No"
        df2.loc[df2["ServicoSegurancaOnline"] == "Yes","ServicosAdicionaisInternet"] = "Yes"
        df2.loc[df2["ServicoBackupOnline"] == "Yes","ServicosAdicionaisInternet"] =  "Yes"
        df2.loc[df2["ServicoProtecaoOnline"] == "Yes","ServicosAdicionaisInternet"] =  "Yes"
        df2.loc[df2["ServicoPctFilmes"] == "Yes","ServicosAdicionaisInternet"] = "Yes"
        df2.loc[df2["ServicoSuporteOnline"] == "Yes","ServicosAdicionaisInternet"] = "Yes"
        df2.loc[df2["ServicoTV"] == "Yes","ServicosAdicionaisInternet"] = "Yes"
        #df2.drop(columns = [ "ServicoSegurancaOnline","ServicoBackupOnline","ServicoSuporteOnline","ServicoProtecaoOnline","ServicoPctFilmes","ServicoTV"],inplace=True)
        df2.drop(columns = ["TemParceiro", "TemDependentes", "ServicoSegurancaOnline","ServicoBackupOnline","ServicoSuporteOnline","ServicoProtecaoOnline","ServicoPctFilmes","ServicoTV"],inplace=True)
        return df2

  def getDataFrameQtdeServicos(self): #80.37
        df2 = self.df.copy()
        df2["QtdeServicos"] = 0
        df2.loc[df2["TemTelefone"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["TemInternet"] != "No","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoTV"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2["QtdeServicos"] = df2["QtdeServicos"].astype('float')
        df2.drop(columns = ["TemTelefone","TemInternet","ServicoTV"],inplace=True)
        return df2          

  def getDataFrameQtdeServicos2(self): 
        df2 = self.df.copy()
        df2["QtdeServicos"] = 0
        df2["QtdeServicosAdicionais"] = 0
        df2.loc[df2["TemTelefone"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["TemInternet"] != "No","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoTV"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoSegurancaOnline"] == "Yes","QtdeServicosAdicionais"] =  df2["QtdeServicosAdicionais"] + 1
        df2.loc[df2["ServicoBackupOnline"] == "Yes","QtdeServicosAdicionais"] =  df2["QtdeServicosAdicionais"] + 1
        df2.loc[df2["ServicoProtecaoOnline"] == "Yes","QtdeServicosAdicionais"] =  df2["QtdeServicosAdicionais"] + 1
        df2.loc[df2["ServicoPctFilmes"] == "Yes","QtdeServicosAdicionais"] =  df2["QtdeServicosAdicionais"] + 1
        df2.loc[df2["ServicoSuporteOnline"] == "Yes","QtdeServicosAdicionais"] =  df2["QtdeServicosAdicionais"] + 1
        df2["QtdeServicos"] = df2["QtdeServicos"].astype('float')
        df2["QtdeServicosAdicionais"] = df2["QtdeServicosAdicionais"].astype('float')
        df2.drop(columns = ["TemTelefone","TemInternet","ServicoTV","ServicoSegurancaOnline","ServicoBackupOnline","ServicoSuporteOnline","ServicoProtecaoOnline","ServicoPctFilmes"],inplace=True)
        return df2          


In [78]:
df_train = TrataDataframe(df_train).getDataFrameDefault()
df_train.head()

,id,gender,SeniorCitizen,Partner,Dependents,tenure,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn
0,4030,Female,0,No,No,56,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,MISS,45,0
1,6731,Male,0,Yes,Yes,16,No,No,No,No,No,No,No,No,One year,No,Mailed check,19,0
2,6479,Female,0,Yes,No,60,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110,0
3,6861,Female,0,No,No,37,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101,1
4,3266,Male,0,Yes,Yes,29,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70,1


In [79]:
df_test = TrataDataframe(df_test).getDataFrameDefault()
df_test.head()

,id,gender,SeniorCitizen,Partner,Dependents,tenure,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges
0,5027,Male,0,Yes,Yes,23,No,No,No,No,No,No,No,No,One year,Yes,Credit card (automatic),20
1,1733,Male,1,Yes,Yes,61,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,One year,No,MISS,99
2,5384,Male,0,No,No,36,Yes,Fiber optic,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,84
3,6554,Female,0,Yes,Yes,61,No phone service,DSL,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),61
4,364,Female,0,No,No,47,No,No,No,No,No,No,No,No,Two year,Yes,Mailed check,20


In [80]:
df_train[['Partner','Dependents']].value_counts()

Partner  Dependents
No       No            2624
Yes      Yes           1345
         No            1278
No       Yes            278
Yes      MISS           109
dtype: int64

In [ ]:
df_train[['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']].value_counts()

In [ ]:
mostra_missing_value(df_train)

In [ ]:
mostra_missing_value(df_test)

In [31]:
df_train['PaymentMethod'].value_counts()

Electronic check             1887
Mailed check                 1272
Bank transfer (automatic)    1212
Credit card (automatic)      1164
MISS                           99
Name: PaymentMethod, dtype: int64

## 7. Separação do dataframe de treino (train.csv) entre preditoras/target

In [83]:
preditoras = df_train.copy()
preditoras.drop(columns=["Churn","id"],inplace=True)
target = df_train["Churn"]
categorical_features_indices = np.where(preditoras.dtypes != np.float)[0] #considerei todas as features que não são do tipo "flutuante" como categóricas
print(f"Qtde de features categóricas: {len(categorical_features_indices)}")
print(f"Colunas preditoras: {preditoras.columns}")

acc = treina_testa(mostrarFI=True)
print(f"acc: {acc}")

Qtde de features categóricas: 17
Colunas preditoras: Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges'],
      dtype='object')
(30.812303550076734, 'Contract')
(18.047672152735153, 'tenure')
(15.299143387979703, 'InternetService')
(7.36237443501442, 'MonthlyCharges')
(6.376566939584991, 'PaymentMethod')
(4.579541509191807, 'MultipleLines')
(3.676945589205436, 'Dependents')
(3.6347787497356907, 'OnlineSecurity')
(3.483656448594219, 'PaperlessBilling')
(2.5993541897029893, 'StreamingTV')
(2.407167498984345, 'StreamingMovies')
(0.7930787794673679, 'TechSupport')
(0.5028382289720654, 'SeniorCitizen')
(0.30314910357612895, 'Partner')
(0.07201436176376627, 'OnlineBackup')
(0.04941507541516035, 'gender')
(0.0, 'DeviceProtection')
acc: 81.08


In [40]:
for seed in range(0,100):
    max_acc = 0
    acc = treina_testa(semente=seed)
    if (acc > max_acc):
        max_acc = acc
        max_seed = seed

print(f'Semente: {max_seed}')
print(f'Acurácia: {max_acc}')

Semente: 99
Acurácia: 82.32


## Transformando variáveis categóricas com `LabelEncoder`

In [84]:
from sklearn.preprocessing import LabelEncoder

for cat_var in df_train.select_dtypes(include='O').columns:
    le = LabelEncoder()
    le.fit(df_train[cat_var])
    df_train[cat_var + '_num'] = le.transform(df_train[cat_var])
    df_train.drop(cat_var, axis=1, inplace=True)
    df_test[cat_var + '_num'] = le.transform(df_test[cat_var])
    df_test.drop(cat_var, axis=1, inplace=True)

df_train.head()

,id,SeniorCitizen,tenure,MonthlyCharges,Churn,gender_num,Partner_num,Dependents_num,MultipleLines_num,InternetService_num,OnlineSecurity_num,OnlineBackup_num,DeviceProtection_num,TechSupport_num,StreamingTV_num,StreamingMovies_num,Contract_num,PaperlessBilling_num,PaymentMethod_num
0,4030,0,56,45,0,0,0,1,1,0,1,1,1,1,0,0,2,0,3
1,6731,0,16,19,0,1,1,2,0,2,0,0,0,0,0,0,1,0,4
2,6479,0,60,110,0,0,1,1,2,1,0,1,1,1,1,1,2,1,1
3,6861,0,37,101,1,0,0,1,2,1,0,1,1,0,1,1,0,1,0
4,3266,0,29,70,1,1,1,2,0,1,0,0,0,0,0,0,0,1,2


In [85]:
df_test.head()

,id,SeniorCitizen,tenure,MonthlyCharges,gender_num,Partner_num,Dependents_num,MultipleLines_num,InternetService_num,OnlineSecurity_num,OnlineBackup_num,DeviceProtection_num,TechSupport_num,StreamingTV_num,StreamingMovies_num,Contract_num,PaperlessBilling_num,PaymentMethod_num
0,5027,0,23,20,1,1,2,0,2,0,0,0,0,0,0,1,1,1
1,1733,1,61,99,1,1,2,2,1,0,1,0,0,1,1,1,0,3
2,5384,0,36,84,1,0,1,2,1,0,0,0,0,0,1,0,1,2
3,6554,0,61,61,0,1,2,1,0,0,1,1,1,1,1,2,1,0
4,364,0,47,20,0,0,1,0,2,0,0,0,0,0,0,2,1,4


##9. Executa a função treina_testa diversas vezes (10x), para as diferentes combinações de test_size, it, lr e depth,  armazenando o resultado de cada execução em um dicionário para análise posterior.

In [91]:
l_ts = [0.18, 0.20, 0.22, 0.24, 0.25, 0.26, 0.28]
l_it = [300, 400, 500]
l_lr = [0.02, 0.03, 0.04]
l_depth = [2, 3, 4, 5]
#l_ts, l_it, l_lr, l_depth = [0.22], [300], [0.04], [4]
resultado = {}

for vez in range(1,11):
   resultado[vez] = {"ts":[], "it":[], "lr":[],"depth":[],"score":[]}
   for ts in l_ts:
       print(f'execução {vez}/ts {ts}...')
       for it in l_it:
           for lr in l_lr:
               for depth in l_depth:
                  res = treina_testa(ts=ts, it=it, lr=lr, depth=depth, gerarArquivo=False, mostrarFI=False) #não vai salvar o arquivo e nem mostrar as melhores features
                  resultado[vez]["ts"].append(ts)
                  resultado[vez]["it"].append(it)
                  resultado[vez]["lr"].append(lr)
                  resultado[vez]["depth"].append(depth)
                  resultado[vez]["score"].append(res)
 

execução 1/ts 0.22...
execução 1/ts 0.24...
execução 1/ts 0.25...
execução 1/ts 0.26...
execução 1/ts 0.28...
execução 1/ts 0.3...
execução 1/ts 0.32...
execução 2/ts 0.22...
execução 2/ts 0.24...
execução 2/ts 0.25...
execução 2/ts 0.26...
execução 2/ts 0.28...
execução 2/ts 0.3...
execução 2/ts 0.32...
execução 3/ts 0.22...
execução 3/ts 0.24...
execução 3/ts 0.25...
execução 3/ts 0.26...
execução 3/ts 0.28...
execução 3/ts 0.3...
execução 3/ts 0.32...


KeyboardInterrupt: ignored

## 10. Transforma o dicionário gerado no passo anterior em um Dataframe e apresenta seus resultados organizados, permitindo verificar qual conjunto (ts,it, lr e depth) gerou boa média de nota e pequeno std.


In [92]:
arrays = []
for vez, v in resultado.items():
    #print(v)
    d = pd.DataFrame(v)
    d["vez"] = vez
    arrays.append(d.copy())
dfResultado = pd.concat(arrays,axis=0,sort=False)

dfTotais = dfResultado.groupby(by=["ts","it","lr","depth"]).agg({"score":["min","max","mean","std"], "vez": ["count"]}).sort_values([("score","mean")],ascending=False)
dfTotais.head(10)

score                               vez
                       min    max       mean       std count
ts   it  lr   depth                                         
0.30 300 0.02 3      81.37  83.80  82.240000  1.353994     3
0.26 400 0.02 4      81.09  83.00  82.090000  0.958175     3
0.22 500 0.03 4      81.13  82.50  81.856667  0.688791     3
0.26 500 0.04 2      80.89  82.66  81.796667  0.885795     3
0.24 500 0.02 5      80.04  82.71  81.720000  1.462635     3
0.25 500 0.02 5      81.26  82.61  81.710000  0.779423     3
0.28 300 0.03 4      80.42  82.32  81.603333  1.032392     3
0.22 400 0.02 5      80.81  82.42  81.560000  0.810617     3
0.32 500 0.02 3      81.36  81.70  81.530000  0.240416     2
0.22 400 0.04 4      80.81  82.18  81.506667  0.685298     3

In [93]:
dfTotais_max = dfResultado.groupby(by=["ts","it","lr","depth"]).agg({"score":["min","max","mean","std"], "vez": ["count"]}).sort_values([("score","max")],ascending=False)
dfTotais_max.head(10)

score                               vez
                       min    max       mean       std count
ts   it  lr   depth                                         
0.22 500 0.02 5      79.60  83.87  81.426667  2.200780     3
0.30 300 0.02 3      81.37  83.80  82.240000  1.353994     3
0.25 400 0.03 3      79.42  83.25  81.240000  1.922056     3
0.26 400 0.02 4      81.09  83.00  82.090000  0.958175     3
0.24 500 0.02 5      80.04  82.71  81.720000  1.462635     3
         0.04 2      80.19  82.71  81.423333  1.260846     3
0.25 300 0.02 4      79.63  82.68  81.240000  1.532090     3
0.26 500 0.04 2      80.89  82.66  81.796667  0.885795     3
0.24 500 0.04 4      79.08  82.63  81.150000  1.847079     3
0.25 500 0.02 5      81.26  82.61  81.710000  0.779423     3

In [94]:
dfTotais_std = dfResultado.groupby(by=["ts","it","lr","depth"]).agg({"score":["min","max","mean","std"], "vez": ["count"]}).sort_values([("score","std")],ascending=True)
dfTotais_std.head(10)

score                               vez
                       min    max       mean       std count
ts   it  lr   depth                                         
0.32 500 0.04 2      79.37  79.42  79.395000  0.035355     2
0.30 300 0.03 2      79.66  79.78  79.740000  0.069282     3
     400 0.03 2      80.25  80.43  80.370000  0.103923     3
0.25 400 0.02 4      79.99  80.20  80.106667  0.106927     3
     300 0.02 2      79.70  79.91  79.793333  0.106927     3
     500 0.04 3      80.55  80.77  80.646667  0.112398     3
0.30 300 0.03 3      81.31  81.55  81.450000  0.124900     3
     500 0.03 3      80.37  80.60  80.523333  0.132791     3
0.26 400 0.03 5      80.20  80.48  80.340000  0.140000     3
0.25 300 0.03 5      80.48  80.77  80.650000  0.151327     3

## 11. Executa a função testa_treina 10 vezes, com os melhores parâmetros, gerando os arquivos.

In [60]:
def treina_testa(ts=0.30, it=300, lr=0.03, depth=5, gerarArquivo=False, mostrarFI=False, semente=None):
   X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(preditoras, target, test_size = ts, random_state = semente)
   catb = CatBoostClassifier(iterations=it, learning_rate=lr, depth=depth, silent=True)
   catb_tuned = catb.fit(X_treinamento, y_treinamento, cat_features=categorical_features_indices)
   y_pred = catb_tuned.predict(X_teste)
   acc_catb = round(accuracy_score(y_pred, y_teste) * 100, 2)
   #print('ts: {} it: {} lr: {} depth: {} -> '.format(ts,it,lr,depth)+'Train Data Success Score: %' + str(acc_catb))
   
   if (mostrarFI == True):
      l_fi = list(zip(catb_tuned.feature_importances_,X_treinamento.columns))
      l_fi.sort(reverse=True)
      for tupla in l_fi:
        print(tupla)

   if (gerarArquivo == True):
      df_id = df_test[["id"]]
      df_teste3 = df_test.drop(columns=["id"])
      resposta = catb_tuned.predict(df_teste3)
      resposta_df = pd.DataFrame(resposta, columns=['Churn'])
      resultado_submissao = pd.concat([df_id, resposta_df],axis=1)
      resultado_submissao.head().T
      filename = '/gdrive/My Drive/Python/submissao_kaggle_catb_fs_ts0{}_it{}_lr{}_depth{}_sc{}.csv'.format(round(ts*100,0),it, lr, depth,str(int(acc_catb*100)))
      resultado_submissao.to_csv(filename, index=False)   
      print(filename)
   
   return acc_catb

In [86]:
#melhor resultado:
l_ts, l_it, l_lr, l_depth = [.22], [300], [0.04], [4]
#l_ts, l_it, l_lr, l_depth = [.25], [400], [0.02], [5]
#l_ts, l_it, l_lr, l_depth = [.24], [300], [0.04], [4]

resultado = {}

for vez in range(1,11):
   resultado[vez] = {"ts":[], "it":[], "lr":[],"depth":[],"score":[]}
   for ts in l_ts:
       #print(f'execução {vez}/ts {ts}...')
       for it in l_it:
           for lr in l_lr:
               for depth in l_depth:
                  res = treina_testa(ts=ts, it=it, lr=lr, depth=depth, gerarArquivo=True, mostrarFI=False)
                  resultado[vez]["ts"].append(ts)
                  resultado[vez]["it"].append(it)
                  resultado[vez]["lr"].append(lr)
                  resultado[vez]["depth"].append(depth)
                  resultado[vez]["score"].append(res)
   #print(resultado[vez])   
resultado  

CatBoostError: ignored

## Salvando o dataframe tratado

In [50]:
df_train.to_csv('/gdrive/My Drive/Python/treino_limpo4.csv', index=False)
df_test.to_csv('/gdrive/My Drive/Python/teste_limpo4.csv', index=False)